In [1]:
import sys
sys.path.insert(0, "../..")

from MPyDATA_examples.Olesik_et_al_2020.analysis import compute_figure_data
from MPyDATA_examples.Olesik_et_al_2020.physics.equilibrium_drop_growth import PdfEvolver
from MPyDATA_examples.Olesik_et_al_2020.plotter import Plotter
from MPyDATA_examples.Olesik_et_al_2020.convergence import convergence_in_time, convergence_in_space
from MPyDATA_examples.Olesik_et_al_2020.distributions import n_n, n_s
from MPyDATA_examples.Olesik_et_al_2020.coordinates import x_id, x_p2
from MPyDATA_examples.utils.error_norms import L2
from MPyDATA_examples.utils.show_plot import show_plot

import matplotlib.pyplot as plt
import matplotlib

import numpy as np

In [2]:
opt_set = [({'n_iters': 1},), 
           ({'n_iters': 1},{'n_iters':2},{'n_iters':3}),
           ({'n_iters': 1},{'n_iters':2},{'n_iters':2, 'iga':True}),
          ({'n_iters': 1},{'n_iters':2},{'n_iters':2, 'iga':True, 'fct': True}),
          ({'n_iters': 1},{'n_iters':2},{'n_iters':2, 'iga':True, 'fct': True, 'tot': True})]